In [1]:
import pandas as pd 
import pyodbc 

### Load sales fact data to FactSales table

#### Create a connection to SQL Srever 

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
# create connection string
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DB_SERVER;"
    "Database=DB_NAME;"
    "Trusted_Connection=yes;" 
)

In [3]:
cursor = conn.cursor()

#### Populate FactSales table from the staging table

##### Load data incrementaly 

In [5]:
# Sql query to insert data to the FactSales table from the fact staging table
sql_load_fact_sales = '''
INSERT INTO FactSales(order_number
      ,order_line_number
      ,product_code
      ,date_id
      ,customer_id
      ,quantity
      ,unit_price
      ,sales
      )
      SELECT s.order_number
      ,s.order_line_number
      ,p.product_code
      ,d.date_id
      ,c.customer_id
      ,s.quantity
      ,s.unit_price
      ,s.sales
      FROM stg_sales s
      INNER JOIN DimDate d
      on s.date_id = d.date_id
      INNER JOIN DimProduct p
      on trim(s.product_code) = trim(p.product_code)
      INNER JOIN DimCustomer c
      on trim(s.customer_name) = trim(c.customer_name)
      and trim(s.contact_lastname) = trim (c.contact_lastname)
      and trim(s.contact_firstname) = trim (c.contact_firstname)
      and trim(s.country) = trim(c.country)
      and trim(s.deal_size) = trim(c.deal_size)
      LEFT JOIN FactSales f                     --- check for exising records
      on s.order_number = f.order_number
      and s.order_line_number = f.order_line_number
      WHERE f.order_number is Null          ---- insert new records
'''
cursor.execute(sql_load_fact_sales)
conn.commit()
print("FactSales table successfully populated")

FactSales table successfully populated
